In [7]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image
import pandas as pd

import time

In [4]:
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

True
0
1
Quadro P1000


In [5]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
latent_size = 64
hidden_size = 256
image_size = 784
num_epochs = 200
batch_size = 100
sample_dir = 'samples'


In [6]:
# Create a directory if not exists
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [54]:
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5],   # 1 for greyscale channels
                                     std=[0.5])])

In [55]:
# MNIST dataset
mnist = torchvision.datasets.MNIST(root='../../data/',
                                   train=True,
                                   transform=transform,
                                   download=True)

# Data loader
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size=batch_size, 
                                          shuffle=True)

In [56]:
# Discriminator
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid())

In [57]:
# Generator 
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh())

In [70]:
total_step = len(data_loader)
log_list_per_25_epochs_cpu = []
log_list_per_25_epochs_cuda = []
device_types = ['cpu', 'cuda']

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for device_type in device_types:
    device = torch.device(device_type)
    D = D.to(device)
    G = G.to(device)
    
    print('# ================================================================== #')
    print('#                                      {}                                         #'.format(device_type))
    print('# ================================================================== #')
    
    
    # Binary cross entropy loss and optimizer
    criterion = nn.BCELoss()
    d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
    g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

    def denorm(x):
        out = (x + 1) / 2
        return out.clamp(0, 1)


    def reset_grad():
        d_optimizer.zero_grad()
        g_optimizer.zero_grad()
    
    
    starts = time.time()
    for epoch in range(num_epochs):
        for i, (images, _) in enumerate(data_loader):
            images = images.reshape(batch_size, -1).to(device)

            # Create the labels which are later used as input for the BCE loss
            real_labels = torch.ones(batch_size, 1).to(device)
            fake_labels = torch.zeros(batch_size, 1).to(device)


    # ================================================================== #
    #                      Train the discriminator                       #
    # ================================================================== #

            outputs = D(images)
            d_loss_real = criterion(outputs, real_labels)
            real_score = outputs

            # Compute BCELoss using fake images
            # First term of the loss is always zero since fake_labels == 0
            z = torch.randn(batch_size, latent_size).to(device)
            fake_images = G(z)
            outputs = D(fake_images)
            d_loss_fake = criterion(outputs, fake_labels)
            fake_score = outputs     


            # Backprop and optimize
            d_loss = d_loss_real + d_loss_fake
            reset_grad()
            d_loss.backward()
            d_optimizer.step()


            # ================================================================== #
            #                        Train the generator                         #
            # ================================================================== #

            # Compute loss with fake images
            z = torch.randn(batch_size, latent_size).to(device)
            fake_images = G(z)
            outputs = D(fake_images)

            # We train G to maximize log(D(G(z)) instead of minimizing log(1-D(G(z)))
            # For the reason, see the last paragraph of section 3. https://arxiv.org/pdf/1406.2661.pdf
            g_loss = criterion(outputs, real_labels)

            # Backprop and optimize
            reset_grad()
            g_loss.backward()
            g_optimizer.step()

            if epoch %25 ==0 and (i+1) % 200 == 0:
                print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                      .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                              real_score.mean().item(), fake_score.mean().item()))
                              
            if device_type =='cpu':
                if epoch % 25 == 0 and (i+1) % 600==0:
                    take_time = time.time() - starts
                    log_list_per_25_epochs_cpu.append(take_time)
            
            else:
                if epoch % 25 == 0 and (i+1) % 600==0:
                    take_time = time.time() - starts
                    log_list_per_25_epochs_cuda.append(take_time)

            if epoch % 50 ==0 and (i+1) % 600 ==0:
                take_time = time.time() - starts
                print('{} Computing Time: {:.4f}'.format(device_type, take_time))
            

# ================================================================== #
#                                      cpu                                         #
# ================================================================== #
Epoch [0/200], Step [200/600], d_loss: 0.4160, g_loss: 5.4803, D(x): 0.86, D(G(z)): 0.02
Epoch [0/200], Step [400/600], d_loss: 0.0908, g_loss: 6.8134, D(x): 0.96, D(G(z)): 0.02
Epoch [0/200], Step [600/600], d_loss: 0.0927, g_loss: 6.3092, D(x): 0.97, D(G(z)): 0.03
cpu Computing Time: 26.5900
Epoch [25/200], Step [200/600], d_loss: 0.2319, g_loss: 3.8343, D(x): 0.91, D(G(z)): 0.06
Epoch [25/200], Step [400/600], d_loss: 0.5225, g_loss: 3.0529, D(x): 0.79, D(G(z)): 0.07
Epoch [25/200], Step [600/600], d_loss: 0.6297, g_loss: 3.1705, D(x): 0.82, D(G(z)): 0.13
Epoch [50/200], Step [200/600], d_loss: 0.7602, g_loss: 2.4219, D(x): 0.72, D(G(z)): 0.14
Epoch [50/200], Step [400/600], d_loss: 0.4936, g_loss: 1.5299, D(x): 0.88, D(G(z)): 0.23
Epoch [50/200], Step [600/60

In [74]:
log_list_per_25_epochs_cpu

[26.589967489242554,
 759.5416057109833,
 1548.730007648468,
 2314.4079563617706,
 3078.567747116089,
 3856.698341846466,
 4642.250211954117,
 5438.164389133453]

In [79]:
log_list_per_25_epochs_cuda

[18.09806537628174,
 468.9906985759735,
 911.4684159755707,
 1356.9603612422943,
 1799.7313268184662,
 2243.6638650894165,
 2686.466948032379,
 3138.6666300296783]

In [94]:
df = pd.DataFrame(data={'Epochs': ['1 Time', '25 Times','50 Times', '75 Times','100 Times','125 Times','150 Times','175 Times'],
    'local CPU':[26.589967489242554,  759.5416057109833,
                                     1548.730007648468,  2314.4079563617706,3078.567747116089,
                                     3856.698341846466, 4642.250211954117, 5438.164389133453],
                       'local GPU':[18.09806537628174, 468.9906985759735, 911.4684159755707, 1356.9603612422943, 
                                    1799.7313268184662, 2243.6638650894165, 2686.466948032379, 3138.6666300296783],
                       'Nipa CPU':[75.14369893074036,1666.85045003891, 2370.6793422698975, 2891.0353157520294, 
                                   3403.7471404075623, 3919.007220506668, 4515.387395143509, 5201.6457941532135],
                       'Nipa GPU': [12.760392427444458, 348.51575088500977, 675.844676733017, 1009.0489814281464, 
                                    1342.2661945819855, 1675.650508403778, 2006.4203536510468, 2340.548830986023]})


In [96]:
df = df.round(2)

In [97]:
df

,Epochs,local CPU,local GPU,Nipa CPU,Nipa GPU
0,1 Time,26.59,18.10,75.14,12.76
1,25 Times,759.54,468.99,1666.85,348.52
2,50 Times,1548.73,911.47,2370.68,675.84
3,75 Times,2314.41,1356.96,2891.04,1009.05
4,100 Times,3078.57,1799.73,3403.75,1342.27
5,125 Times,3856.70,2243.66,3919.01,1675.65
6,150 Times,4642.25,2686.47,4515.39,2006.42
7,175 Times,5438.16,3138.67,5201.65,2340.55
